In [1]:
!pip install pandas mistralai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [2]:
import pandas as pd

data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

df = pd.DataFrame(data)

In [3]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps({'status': df[df.transaction_id == transaction_id].payment_status.item()})
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps({'date': df[df.transaction_id == transaction_id].payment_date.item()})
    return json.dumps({'error': 'transaction id not found.'})

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Get payment status of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Get payment date of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    }
]


In [5]:
import functools

names_to_functions = {
  'retrieve_payment_status': functools.partial(retrieve_payment_status, df=df),
  'retrieve_payment_date': functools.partial(retrieve_payment_date, df=df)
}

In [15]:
from mistralai.models.chat_completion import ChatMessage

messages = [
    ChatMessage(role="user", content="What's the status of my transaction T1004?"),
    ChatMessage(role="user", content="What's the date of my transaction T1005")
]


In [16]:
from mistralai.client import MistralClient

model = "mistral-small-latest"
api_key="rrXMIjg4zKH2qCYv7jRPVVgYYuMMbfnt"

client = MistralClient(api_key=api_key)

response = client.chat(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

response

ChatCompletionResponse(id='16b0fcd543f34bcdb8f7add1221ac74b', object='chat.completion', created=1721316104, model='mistral-small-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='SvN7F1vBV', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1004"}'))], tool_call_id=None), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=181, total_tokens=211, completion_tokens=30))

In [17]:
messages.append(response.choices[0].message)

In [18]:
messages

[ChatMessage(role='user', content="What's the status of my transaction T1004?", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content="What's the date of my transaction T1005", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='SvN7F1vBV', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1004"}'))], tool_call_id=None)]

In [19]:
import json
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_payment_status 
function_params:  {'transaction_id': 'T1004'}


In [20]:
function_result = names_to_functions[function_name](**function_params)
function_result


'{"status": "Paid"}'

In [21]:
messages.append(ChatMessage(role="tool", name=function_name, content=function_result, tool_call_id=tool_call.id))

In [22]:
messages

[ChatMessage(role='user', content="What's the status of my transaction T1004?", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='user', content="What's the date of my transaction T1005", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='SvN7F1vBV', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1004"}'))], tool_call_id=None),
 ChatMessage(role='tool', content='{"status": "Paid"}', name='retrieve_payment_status', tool_calls=None, tool_call_id='SvN7F1vBV')]

In [23]:
response = client.chat(
    model=model,
    messages=messages
)

response.choices[0].message.content

'Your transaction T1004 has been paid.\n\n[{"name": "retrieve_transaction_date", "arguments": {"transaction_id": "T1005"}, "id": "881Yj4X4n"}]'